In [1]:
import torch
import tenseal as ts
import pandas as pd
import random
from time import time
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import csv
import warnings
warnings.filterwarnings("ignore") 

In [2]:
df = pd.read_csv("diabetes.csv")
df



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [4]:
y = df['Outcome']
X = df.drop('Outcome', axis=1)
X.shape

(768, 8)

In [7]:
## Splitting for X and Y variables:
## Splitting dataset into 80% Training and 20% Testing Data:
import random
import sklearn
X_train = []
X_test = []
y_train = []
y_test = []
for i in len(X):
    rand = random.random()
    if rand > 0.2:
        X_train.append(X[i])
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state =0)



TypeError: 'int' object is not iterable

In [ ]:
X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(X_test.values).float()
y_train = torch.from_numpy(y_train.values).long()
y_test = torch.from_numpy(y_test.values).long()

In [ ]:
class MLP(nn.Module):
    def __init__(self, n_i, n_o):
        super(MLP, self).__init__()
        self.linear = nn.Linear(n_i, n_o)
        
    def forward(self, x):
        x = self.linear(x)
        return x


In [ ]:
num_epochs = 10000
model = MLP(X_train.shape[1], y_train.unique().size()[0])

In [ ]:
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
def train():
    loss_mean = 0.0
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        loss.backward()
        optimizer.step()
        loss_mean += loss.item()
        if (epoch+1) % 100 == 0:                                         
            print(f'epoch: {epoch+1}, loss = {loss_mean / 100:.4f}')
            loss_mean = 0.0

train()

epoch: 100, loss = 4.2837
epoch: 200, loss = 2.2890
epoch: 300, loss = 1.5422
epoch: 400, loss = 0.9820
epoch: 500, loss = 0.7318
epoch: 600, loss = 0.6411
epoch: 700, loss = 0.6161
epoch: 800, loss = 0.6085
epoch: 900, loss = 0.6030
epoch: 1000, loss = 0.5977
epoch: 1100, loss = 0.5924
epoch: 1200, loss = 0.5873
epoch: 1300, loss = 0.5823
epoch: 1400, loss = 0.5775
epoch: 1500, loss = 0.5728
epoch: 1600, loss = 0.5682
epoch: 1700, loss = 0.5638
epoch: 1800, loss = 0.5596
epoch: 1900, loss = 0.5555
epoch: 2000, loss = 0.5515
epoch: 2100, loss = 0.5478
epoch: 2200, loss = 0.5442
epoch: 2300, loss = 0.5407
epoch: 2400, loss = 0.5374
epoch: 2500, loss = 0.5342
epoch: 2600, loss = 0.5312
epoch: 2700, loss = 0.5283
epoch: 2800, loss = 0.5256
epoch: 2900, loss = 0.5230
epoch: 3000, loss = 0.5205
epoch: 3100, loss = 0.5181
epoch: 3200, loss = 0.5158
epoch: 3300, loss = 0.5137
epoch: 3400, loss = 0.5117
epoch: 3500, loss = 0.5097
epoch: 3600, loss = 0.5079
epoch: 3700, loss = 0.5062
epoch: 380

In [ ]:
with torch.no_grad():
    logits = model(X_test)
    #y_pred = nn.Softmax(dim=1)(logits)
    y_predicted_cls = logits.argmax(1)
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.8247


In [ ]:
print(model.linear.weight.data.tolist()[0])
print(model.linear.weight.data.tolist()[1])
print(model.linear.bias.data.tolist())

[-0.12246307730674744, 0.06274420022964478, -0.05025584623217583, 0.04181673377752304, 0.1361420452594757, -0.05460747331380844, -0.4560495615005493, -0.12040003389120102]
[-0.03104415349662304, 0.09664390236139297, -0.0640219897031784, 0.04624515026807785, 0.13472746312618256, 0.03315586969256401, 0.43858933448791504, -0.10050632804632187]
[4.125000953674316, -4.053066253662109]


In [ ]:
class EnprMLP:
    def __init__(self, model):
        self.weight = model.linear.weight.data.tolist()
        self.bias = model.linear.bias.data.tolist()
    def forward(self, enc_x):
        enc_out = []
        for i in range(len(self.bias)):
            enc_out.append(enc_x.dot(self.weight[i]) + self.bias[i])
        return enc_out
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)
    def decrypt(self):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()

enprmlp = EnprMLP(model)

In [ ]:
poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_eval.global_scale = 2 ** 20
ctx_eval.generate_galois_keys()

In [ ]:
t_start = time()
enc_X_test = [ts.ckks_vector(ctx_eval, x.tolist()) for x in X_test]
t_end = time()
print(f"Encryption of the test-set took {int(t_end - t_start)} seconds")

Encryption of the test-set took 0 seconds


In [ ]:
def enc_evaluation(model, enc_X_test, y_test):
    t_start = time()
    correct = 0
    for enc_x, y in zip(enc_X_test, y_test):
        # encrypted evaluation
        enc_out = model(enc_x)
        # plain comparaison
        out = []
        for x in enc_out:
            out.append(x.decrypt())
        y_predicted_cls = np.array(out).argmax()
        if y == y_predicted_cls:
            correct += 1
    

    t_end = time()
    print(f"Evaluated test_set of {len(y_test)} entries in {int(t_end - t_start)} seconds")
    print(f'accuracy: {(correct / len(y_test)):.4f}')

enc_evaluation(enprmlp, enc_X_test, y_test)

Evaluated test_set of 154 entries in 0 seconds
accuracy: 0.8052


In [ ]:
class EntrMLP:
    def __init__(self, model):
        self.weight = model.linear.weight.data.tolist()
        self.bias = model.linear.bias.data.tolist()
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0
    def forward(self, enc_x):
        enc_out = []
        for i in range(len(self.bias)):
            enc_out.append(enc_x.dot(self.weight[i]) + self.bias[i])
        return self.softmax(enc_out)
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    def encrypt(self, context):
        self.weight = [ts.ckks_vector(context, w) for w in self.weight]
        self.bias = [ts.ckks_vector(context, [b]) for b in self.bias]
    def decrypt(self):
        self.weight = [w.decrypt() for w in self.weight]
        self.bias = [b.decrypt()[0] for b in self.bias]
    def exp(self, enc_x):
        return enc_x.polyval([1, 1])
        #return enc_x.polyval([1, 1, 1/2, 1/6, 1/24, 1/120])
    def ln(self, enc_x):
        enc_x -= 1
        return enc_x.polyval([0, 1])
        #return enc_x.polyval([0, 1, -1/2, 1/3, -1/4, 1/5])
    def softmax(self, enc_x):
        return (enc_x[1] - enc_x[0]).polyval([0.5, 0.197, 0, -0.004])
    def backward(self, enc_x, enc_out, enc_y):
        out_minus_y = (enc_out - enc_y)
        self._delta_w += enc_x * out_minus_y
        self._delta_b += out_minus_y
        self._count += 1
    def update_parameters(self):
        if(self._count == 0):
            raise RuntimeError("You should at least run one forward iteration")
        self.weight[1] -= self._delta_w * (1 / self._count)
        self.bias[1] -= self._delta_b * (1 / self._count)
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0
    
    
        

In [ ]:
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
ctx_tr = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_tr.global_scale = 2 ** 21
ctx_tr.generate_galois_keys()

In [ ]:
print(type(X_train[0].tolist()))
print(type([y_train[0].tolist()]))

<class 'list'>
<class 'list'>


In [ ]:
t_start = time()
enc_X_train = [ts.ckks_vector(ctx_tr, x.tolist()) for x in X_train]
enc_y_train = [ts.ckks_vector(ctx_tr, [y.tolist()]) for y in y_train]
t_end = time()
print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")

Encryption of the training_set took 10 seconds


In [ ]:
entrmlp = EntrMLP(model)
entrmlp.encrypt(ctx_tr)
accuracy = 0
times = []
for epoch in range(1):
    t_start = time()
    for enc_x, enc_y in zip(enc_X_train, enc_y_train):
        enc_out = entrmlp.forward(enc_x)
        entrmlp.backward(enc_x, enc_out, enc_y)
    entrmlp.update_parameters()
    t_end = time()
    times.append(t_end - t_start)

print(f"\nAverage time per epoch: {int(sum(times) / len(times))} seconds")



Average time per epoch: 63 seconds


In [ ]:
def evaluation(model, enc_X_test, y_test):
    t_start = time()
    correct = 0
    for enc_x, y in zip(enc_X_test, y_test):
        # encrypted evaluation
        out = model(enc_x).decrypt()
        print(y)
        print(out[0])
        if abs(y - out[0]) < 0.5:
            correct += 1
    

    t_end = time()
    print(f"Evaluated test_set of {len(y_test)} entries in {int(t_end - t_start)} seconds")
    print(f'accuracy: {(correct / len(y_test)):.4f}')

In [ ]:
entrmlp.decrypt()
entrmlp.encrypt(ctx_tr)
enc_X_test = [ts.ckks_vector(ctx_tr, x.tolist()) for x in X_test]
evaluation(entrmlp, enc_X_test, y_test)

tensor(1)
1030278498280.4307
tensor(0)
665585283372.8848
tensor(0)
89735011852.45557
tensor(1)
2359371404371.2783
tensor(0)
210048474311.1577
tensor(0)
205661376525.98605
tensor(1)
2036134581141.7969
tensor(1)
2473640026045.767
tensor(0)
2701918766682.0874
tensor(0)
1459253745699.1123
tensor(1)
3232877916665.942
tensor(1)
3597638943608.3716
tensor(0)
271601086829.07074
tensor(0)
466340368028.04675
tensor(0)
89729206122.71912
tensor(0)
219611380843.20102
tensor(1)
2058347308919.2969
tensor(0)
70735462643.14392
tensor(0)
452922567060.08856
tensor(0)
472936396119.483
tensor(1)
500321490409.5231
tensor(1)
270140076905.46887
tensor(0)
81581094949.03813
tensor(0)
92797529933.1774
tensor(0)
744942039554.8658
tensor(0)
293417113613.16675
tensor(0)
182877304146.99838
tensor(0)
1799667110787.1511
tensor(0)
224553615853.93546
tensor(0)
209711908203.09448
tensor(0)
1362096648868.021
tensor(0)
508676279800.5748
tensor(0)
659782899444.5242
tensor(0)
1262731291328.2493
tensor(0)
415648166544.9888
ten